In [1]:
!pip install "accelerate>=0.16.0,<1" "transformers[torch]>=4.28.1,<5" "torch>=1.13.1,<2" faiss-cpu
!pip install PyPDF2
!pip install langchain
!pip install transformers
!pip install sentence_transformers

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 2.4 MB/s eta 0:00:00a 0:00:01
  Using cached torch-1.13.1-cp310-cp310-manylinux1_x86_64.whl (887.5 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 3.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 2.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 5.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 3.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0
  Attempting uninstall: torch
    Found existing installation: torch 2.1.0
    Uninstalling torch-2.1.0:
      Successfully uninstalled torch-2.1.0
  Attempting uninstall: tokenizers
    Found existing installation: t

In [4]:
import transformers
import PyPDF2
import re
from sentence_transformers import SentenceTransformer
import faiss
from transformers import pipeline, AutoTokenizer
from transformers import AutoModelForSeq2SeqLM

In [14]:
model_name='google/flan-t5-base'
tokenizer=AutoTokenizer.from_pretrained(model_name,use_fast=True)

model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")


with open("U-Net.pdf", "rb") as pdf_file:
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()
print(text)

# cleaned_text = re.sub(r"\s+", " ", text).strip()

model_sbert = SentenceTransformer("all-mpnet-base-v2")
# embeddings = model_sbert.encode(cleaned_text.split("."))
embeddings = model_sbert.encode(text.split("."))

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)


U-Net: Convolutional Networks for Biomedical
Image Segmentation
Olaf Ronneberger, Philipp Fischer, and Thomas Brox
Computer Science Department and BIOSS Centre for Biological Signalling Studies,
University of Freiburg, Germany
ronneber@informatik.uni-freiburg.de ,
WWW home page: http://lmb.informatik.uni-freiburg.de/
Abstract. There is large consent that successful training of deep net-
works requires many thousand annotated training samples. In this pa-
per, we present a network and training strategy that relies on the strong
use of data augmentation to use the available annotated samples more
eciently. The architecture consists of a contracting path to capture
context and a symmetric expanding path that enables precise localiza-
tion. We show that such a network can be trained end-to-end from very
few images and outperforms the prior best method (a sliding-window
convolutional network) on the ISBI challenge for segmentation of neu-
ronal structures in electron microscopic stacks. Us

In [26]:
def answer_question(question):
    query_embedding = model_sbert.encode([question])
    distances, indices = index.search(query_embedding, k=5)

    retrieved_text = [text.split(".")[i] for i in indices.flatten()]
    retrieved_text_combined = " ".join(retrieved_text)
    # print(retrieved_text_combined)
    input_ids = tokenizer(retrieved_text_combined, return_tensors="pt")
    # print("Input IDs:", input_ids)
    answer = model.generate(**input_ids,max_length=50)
    return answer 

In [27]:
question = "explain Data Augmentation"
answer = answer_question(question)
answer_text = tokenizer.decode(answer[0], skip_special_tokens=True)  
print(f"Answer to question '{question}': {answer_text}")

Answer to question 'explain Data Augmentation': We propose a network and training strategy that relies on the strong use of data augmentation to use the available annotated samples more eciently.
